In [ ]:
from langchain_community.document_loaders import Docx2txtLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500, # 청크 하나가 가질 수 있는 토큰 수
    chunk_overlap=200, # 청크 사이즈의 20% 만큼 중복. 왜 중복필요? => 유사도 검색 시 원하는 답변이 나올 수 있는 확률을 높이기 위함 (문맥 유지)
)

loader = Docx2txtLoader("tax_with_markdown.docx") # 문서 읽기 (tax.docx에서 변경됨)
document_list = loader.load_and_split(text_splitter=text_splitter) # 문서 쪼개기

In [49]:
# 임베딩
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings

load_dotenv()
embedding = OpenAIEmbeddings(model="text-embedding-3-large")

In [ ]:
%pip install --upgrade langchain_pinecone langchain-openai langchain

In [50]:
import os

from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore

index_name = "tax-index-markdown" # tax-index 대신 tax-index-markdown으로 변경
pinecone_api_key = os.environ.get("PINECONE_API_KEY")
pinecone_api_key

pc = Pinecone(api_key=pinecone_api_key)

database = PineconeVectorStore.from_documents(
    documents=document_list,
    embedding=embedding,
    index_name=index_name,
    pinecone_api_key=pinecone_api_key,
)


In [51]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
# retrieved_docs = database.similarity_search(query, k=3) # 유사도 검색

In [52]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o")

In [8]:
%pip install -U langchain langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 26.0.1
[notice] To update, run: c:\Users\hanjo\.pyenv\pyenv-win\versions\3.10.11\python.exe -m pip install --upgrade pip


In [53]:
from langchain_classic import hub # langchain은 deprecated 됨

prompt = hub.pull('rlm/rag-prompt')


In [54]:
from langchain_classic.chains import RetrievalQA

retriever = database.as_retriever()
# retriever 결과 원하는 답변을 받아오지 못함. 왜? 이미지 파싱을 못했기 떄문. 이미지를 markdown으로 변경해야한다.
# (tax_with_markdown.docx 참고. ChatGpt의 경우 이미지를 markdown으로 변경해야 읽을 수 있다.)
retriever.invoke(query)

[Document(id='e268a648-a965-44eb-935b-ad9c2949b5bf', metadata={'source': 'tax_with_markdown.docx'}, page_content='나. 그 밖의 배당소득에 대해서는 100분의 14\n\n3. 원천징수대상 사업소득에 대해서는 100분의 3. 다만, 외국인 직업운동가가 한국표준산업분류에 따른 스포츠 클럽 운영업 중 프로스포츠구단과의 계약(계약기간이 3년 이하인 경우로 한정한다)에 따라 용역을 제공하고 받는 소득에 대해서는 100분의 20으로 한다.\n\n4. 근로소득에 대해서는 기본세율. 다만, 일용근로자의 근로소득에 대해서는 100분의 6으로 한다.\n\n5. 공적연금소득에 대해서는 기본세율\n\n5의2.제20조의3제1항제2호나목 및 다목에 따른 연금계좌 납입액이나 운용실적에 따라 증가된 금액을 연금수령한 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 각 목의 요건을 동시에 충족하는 때에는 낮은 세율을 적용한다.\n\n가. 연금소득자의 나이에 따른 다음의 세율\n\n\n\n나. 삭제<2014. 12. 23.>\n\n다. 사망할 때까지 연금수령하는 대통령령으로 정하는 종신계약에 따라 받는 연금소득에 대해서는 100분의 4\n\n5의3. 제20조의3제1항제2호가목에 따라 퇴직소득을 연금수령하는 연금소득에 대해서는 다음 각 목의 구분에 따른 세율. 이 경우 연금 실제 수령연차 및 연금외수령 원천징수세율의 구체적인 내용은 대통령령으로 정한다.\n\n가. 연금 실제 수령연차가 10년 이하인 경우: 연금외수령 원천징수세율의 100분의 70\n\n나. 연금 실제 수령연차가 10년을 초과하는 경우: 연금외수령 원천징수세율의 100분의 60\n\n6. 기타소득에 대해서는 다음에 규정하는 세율. 다만, 제8호를 적용받는 경우는 제외한다.\n\n가. 제14조제3항제8호라목 및 마목에 해당하는 소득금액이 3억원을 초과하는 경우 그 초과하는 분에 대해서는 100분의 30\n\n나. 제21조제1항제

In [45]:
# LangChain 형태로 질문 답변 체인 생성
qa_chain = RetrievalQA.from_chain_type(llm, retriever=database.as_retriever(), chain_type_kwargs={"prompt": prompt})

In [46]:
ai_message = qa_chain({"query": query})

In [47]:
ai_message['result']

'연봉 5천만 원인 직장인의 소득세를 정확히 계산하기 위해서는 소득 구간별 세율표와 각종 공제를 고려해야 합니다. 일반적으로 소득세는 과세 소득의 세율 구간에 따라 계산되며, 원천징수 방식으로 세율이 적용됩니다. 기본적인 계산을 위해 정확한 소득세율과 공제 항목을 확인해야 하므로, 이에 대한 정보는 세무 전문가에게 문의하는 것이 좋습니다.'